# Download data for events
This part of the code will be used to download event data.

It is based on Josh Russell's **fetch_EVENTS**.

*william b hawley april 2020*

In [ ]:
from config import *
import os

import numpy as np
import matplotlib.pyplot as plt

import obspy
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from obspy.geodetics import gps2dist_azimuth, locations2degrees
from obspy.io.sac import SACTrace

from datetime import datetime
import calendar

%matplotlib inline
%load_ext autoreload
%autoreload

In [2]:
# Check to see if data directory exists
if not os.path.exists(DataDir):
    os.makedirs(DataDir)

# load the client
client = Client(webservice)

In [3]:
# Load event catalogue
t1 = UTCDateTime(tstart)
t2 = UTCDateTime(tend)
catIRIS = client.get_events(starttime=t1, endtime=t2, minmagnitude=minMag)

# Load stations
inventoryX = client.get_stations(network=network, station=','.join(XStaList), channel=','.join(XChanList), starttime=t1, endtime=t2)
inventoryD = client.get_stations(network=network, station=','.join(DStaList), channel=','.join(DChanList), starttime=t1, endtime=t2)
inventory = client.get_stations(network=network, station=','.join(StaList), channel=','.join(ChanList), starttime=t1, endtime=t2)
#inventoryAll.plot(projection='global',label=False)
#fig = inventoryAll.plot(method='basemap',show=False)
#catIRIS.plot(method="cartopy")

In [8]:
# Get event times
f = open(EventsFileName,'w')
for iev in range(0,len(catIRIS)):
    if isCMT_params: 
        if isCentroid:
            ior = 1
        else:
            ior = 0
        
        # if searching CMT catalogue, pull out necessary time info
        time = catIRIS[iev].origins[0].time
        date = datetime.strptime(str(time),'%Y-%m-%dT%H:%M:%S.%fZ')
        month = calendar.month_abbr[date.month].lower()
        year = str(date.year)
        
        # in case time is not exactly the same...
        time1 = str(time-50) #+/- 50 seconds
        time2 = str(time+50)
        catCMT = obspy.read_events("https://www.ldeo.columbia.edu/~gcmt/projects/CMT/catalog/NEW_MONTHLY/"+year+"/"+month+""+year[2:4]+".ndk")
        cat_filt = catCMT.filter('time > '+time1, 'time < '+time2, 'magnitude >= '+str(minMag-1))
        if len(cat_filt)==0:
            print('Cannot find event in GCMT catalogue... using IRIS')
            cat = catIRIS[iev].copy()
            ior = 0
        else:
            cat = cat_filt[0].copy()
    else:
        cat = catIRIS[iev].copy()
        ior = 0
    
    # earthquake parameters
    evT1 = cat.origins[ior].time
    evT2 = evT1 + trLen
    evdp = cat.origins[ior].depth
    evla = cat.origins[ior].latitude
    evlo = cat.origins[ior].longitude
    mag = cat.magnitudes[0].mag
    
    # date for naming folders
    date = datetime.strptime(str(evT1),'%Y-%m-%dT%H:%M:%S.%fZ')
    evName = date.strftime('%Y%m%d%H%M')
    evDir = DataDir + evName + '/'
    f.write(evName+'\n')
    if not os.path.exists(evDir):
        os.makedirs(evDir)
        print('Working on event '+evName)
    else:
        print('Event '+evName+' exists, skipping...')
        
    # loop through stations
    #for ista in range(0,len(inventoryX[0])):
    for ista in range(0,len(inventory[0])):
        stel = inventory[0].stations[ista].elevation
        stla = inventory[0].stations[ista].latitude
        stlo = inventory[0].stations[ista].longitude
        station = inventory[0].stations[ista].code
        vals = gps2dist_azimuth(lat1=stla, lon1=stlo, lat2=evla, lon2=evlo)
        dist = vals[0]
        baz = vals[1]
        az = vals[2]
        gcarc = locations2degrees(lat1=stla, long1=stlo, lat2=evla, long2=evlo)
        
        # Loop through components... 
        for comp in ChanList:
            if comp == 'BDH':
                if station in XStaList:
                    comp = 'BXH'
                    print("Using comp "+comp+" for station "+station)
            
            try:
                st = client.get_waveforms(network=network, station=station, location='*', channel=comp, starttime=evT1, endtime=evT2, attach_response=True)
            except Exception:
                print('Missing data for station: ',station)
                continue
                
            # if BXH, change back ... doesn't matter; we downsample to 1 Hz anyway
            if comp == 'BXH':
                comp = 'BDH'
                
            # check for gaps
            if len(st) > 1:
                st.merge(method=1, fill_value=0)
            sr = st[0].stats.sampling_rate
            st.remove_response(output="DISP", zero_mean=True, taper=True, taper_fraction=0.05, pre_filt=[0.001,0.005,sr/3,sr/2], water_level=60)
            st.trim(starttime=evT1, endtime=evT2, pad=True, nearest_sample=False, fill_value=0)
            st.detrend(type='demean')
            st.detrend(type='linear')
            st.taper(type='cosine',max_percentage=0.05)
            
            # downsample the trace
            if isDownsamp==1:
                st.filter('lowpass', freq=0.4*srNew, zerophase=True) #anti-alias
                st.resample(sampling_rate=srNew)
                st.detrend(type='demean')
                st.detrend(type='linear')
                st.taper(type='cosine',max_percentage=0.05)
                
            # convert to SAC
            sac = SACTrace.from_obspy_trace(st[0])
            sac.stel = stel
            sac.stla = stla
            sac.stlo = stlo
            sac.evdp = evdp
            sac.evla = evla
            sac.evlo = evlo
            sac.mag = mag
            sac.dist = dist
            sac.az = az
            sac.baz = baz
            sac.gcarc = gcarc
            sac.kcmpnm = comp
            
            sac_out = evDir + evName + '.' + network + '.' + station + '.' + comp + '.sac'
            sac.write(sac_out)
            

Event 201211161812 exists, skipping...
station: BB030, comp: LHZ
station: BB030, comp: LH1
station: BB030, comp: LH2
Using comp BXH for station BB030
station: BB030, comp: BXH
station: BB060, comp: LHZ
station: BB060, comp: LH1
station: BB060, comp: LH2
Using comp BXH for station BB060
station: BB060, comp: BXH
station: BB070, comp: LHZ
station: BB070, comp: LH1
station: BB070, comp: LH2
Using comp BXH for station BB070
station: BB070, comp: BXH
station: BB090, comp: LHZ
station: BB090, comp: LH1
station: BB090, comp: LH2
Using comp BXH for station BB090
station: BB090, comp: BXH
station: BB120, comp: LHZ
station: BB120, comp: LH1
station: BB120, comp: LH2
Using comp BXH for station BB120
station: BB120, comp: BXH
station: BB130, comp: LHZ
station: BB130, comp: LH1
station: BB130, comp: LH2
Using comp BXH for station BB130
station: BB130, comp: BXH
station: BB140, comp: LHZ
station: BB140, comp: LH1
station: BB140, comp: LH2
Using comp BXH for station BB140
station: BB140, comp: BXH
st

['BB030', 'BB060', 'BB070', 'BB090', 'BB120', 'BB130', 'BB140', 'BB150', 'BB170', 'BB180', 'BB200', 'BB240']
